## **Step 1 - keywords Extraction**
***

We have two datasets, one with dream text descriptions:

In [ ]:
from keyword_extractor import read_datasets, extract_and_save_keywords_from_dataframes
from yaml_parser import load_config
config = load_config()
dream_df, keywords_df = read_datasets(config)
dream_df.head()

And another one with interpretations of dreams according to keywords:

In [ ]:
keywords_df

Now, we will use pretrained LLMs in order to extract the given keywords from the keywords dataset , from the dream text description from the dream text dataset.

### **GPT2**
***

In [ ]:
dream_df = extract_and_save_keywords_from_dataframes()

## Step 2 - Summarize interpretations

### Load data and prepare (small) dataset for experimenting

In [ ]:
import pandas as pd
from datetime import datetime
from transformers import pipeline
from utils import  release_all_gpu_memory, save_df_as_pretty_html


In [ ]:
from summarizer import load_causal_model, batch_generate_interpretations

import torch

In [ ]:
exmpl = dream_df[dream_df["text_dream"].str.len()< 300]

In [ ]:
exmpl = exmpl[["text_dream","Dream Symbol"]].sample(5, random_state=45)

In [ ]:
exmpl

In [ ]:
exmpl["Dream Symbol"]

In [ ]:
dataset = []

prmt = """Given dream description, interpret the meaning of the dream. 
Provided also are the dream symbols that appear in the dream and their meanings. 
Use the dream symbols meanings to help you interpret the dream. """.replace("\n", " ")


for i, ex in dream_df.iterrows():
    #print(ex)
    keys = ex["Dream Symbol"].split(",")[:5]
    
    #print(keys)
    syms = keywords_df[keywords_df["Dream Symbol"].isin(keys)]

    descr = syms.apply(lambda r: f' - {r["Dream Symbol"]}:  {r["Interpretation"]}', axis = 1)
    item = {
        "prompt": prmt, 
        "dream": ex["text_dream"],
        "symbols": "\n".join(descr),
        }
    dataset.append(item)
    

dataset = pd.DataFrame(dataset)
dataset


### Summarize with flan-T5-large model

In [ ]:
release_all_gpu_memory()

In [ ]:
# Step 1: Load FLAN-T5 model and tokenizer
model_name = "google/flan-t5-large"
model_name_short = model_name.split("/")[-1]
device = 0 if torch.cuda.is_available() else -1
model, tokenizer = load_causal_model(model_name)

In [ ]:
text2text_generator = pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=1024,           # ✅ allow longer input
        truncation=True,           # ✅ ensure truncation at tokenizer level
        device=device,
    )

In [ ]:
tstp = datetime.now().strftime(r"%y.%m.%d-%H")
result_df = batch_generate_interpretations(dataset, text2text_generator, batch_size=100, max_length=250)


In [ ]:
postproc = lambda out: out["generated_text"].strip()
result_df["interpretation"] = result_df["interpretation"].apply(postproc)


In [ ]:
result_df

In [ ]:
result_df.columns

In [ ]:
model_name_short

In [ ]:
save_df = result_df[['prompt', 'symbols','dream', 'interpretation']]

path = f"output/{model_name_short}_{tstp}"

save_df_as_pretty_html(save_df, path + ".html")

save_df.to_csv(path + ".csv", index = False)

In [ ]:
result_df.interpretation.str.len()

In [ ]:
test = pd.read_csv(path + ".csv")

In [ ]:
test

### Summarize with Mistral model

In [ ]:
from summarizer import load_mistral_4bit_model

In [ ]:
dataset

In [ ]:
release_all_gpu_memory(["model", "tokenizer", "text2text_generator"])


In [ ]:
print("Loading Mistral-7B-Instruct in 4-bit...")

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
model_name_short = model_name.split("/")[-1]

model_family="decoder"
  
# max_new_tokens=256         max_new_tokens=max_new_tokens,

model, tokenizer = load_mistral_4bit_model(model_name)


In [ ]:
print(f"Sample prompt length (characters): {len(sample_prompt)}")
optimal_batch_size = find_max_batch_size(model, tokenizer, sample_prompt)

In [ ]:
model_pipeline = pipeline(
        "text-generation",
        model=model,
        batch_size=4,
        tokenizer=tokenizer,
        max_length = model.config.max_position_embeddings,
        do_sample=False
    )


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
l = len(dataset)
l
batch_size = int(l/1)
batch_size

In [ ]:
from summarizer import PromptFormatter

formatter = PromptFormatter(model_family)

In [ ]:

print("\n🧠 Running interpretations...")
tstp = datetime.now().strftime(r"%y.%m.%d-%H")

result_df = batch_generate_interpretations(dataset, model_pipeline, formatter, batch_size=67)
#print(result_df[["dream", "interpretation"]])


In [ ]:
tok = model_pipeline.tokenizer
tok

In [ ]:
tok.model_max_length

In [ ]:
model.config.max_position_embeddings


## Trash

In [ ]:
tzd = tokenizer.tokenize(sample_prompt, truncation=False, max_length=1024)

In [ ]:
len(tzd)

In [ ]:
postproc = lambda out: out[0]["generated_text"].split("Interpretation:")[-1].strip()
result_df["interpretation"] = result_df["interpretation"].apply(postproc)


In [ ]:
result_df

In [ ]:

save_df = result_df[['prompt', 'symbols','dream', 'interpretation']]

path = f"output/{model_name_short}_{tstp}"
save_df_as_pretty_html(save_df, path + ".html")

save_df.to_csv(path + ".csv")

In [ ]:
dataset["inputs"] = dataset.apply(lambda row: format_input(row["prompt"], row["dream"], row["symbols"]), axis=1)
dataset

In [ ]:
dataset["input_len"] = dataset["inputs"].str.len()
dataset.sort_values("input_len", ascending=False, inplace=True)
dataset

In [ ]:
shorter_dataset = dataset[dataset.input_len<7000]
shorter_dataset

In [ ]:
shorter_dataset.input_len.hist(bins=100)

In [ ]:
save_df_as_pretty_html(dataset[:100], "output/long_inputs.html")